# Sentiment-Analyse Twitter

In [14]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup

from google.colab import drive

In [15]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds
tf.__version__

'2.4.0'

## Preprocessing

In [16]:
# mount google drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!ls "drive/MyDrive/Datasets/NLP/sentiment140_academics"

testdata.csv  training.csv


### load in the data via pandas

In [18]:
# load data
cols = ["label", "id", "date", "query", "user", "text"]

# train data
train_data = pd.read_csv(
    "drive/MyDrive/Datasets/NLP/sentiment140_academics/training.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
    )

In [19]:
train_data.head(3)

,label,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...


In [20]:
# test data
test_data = pd.read_csv(
    "drive/MyDrive/Datasets/NLP/sentiment140_academics/testdata.csv",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
    )

In [21]:
test_data.head(3)

,label,id,date,query,user,text
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."


### cleaning

In [22]:
# drop not needed columns
train_data.drop(
    ["id", "date", "query", "user"],
    axis=1,
    inplace=True)

train_data.columns

Index(['label', 'text'], dtype='object')

In [ ]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    # RegExr
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    tweet = re.sub(r" +", ' ', tweet) # get rid of white spaces in a row
    return tweet

In [ ]:
train_data_clean = [clean_tweet(tweet) for tweet in train_data.text]

In [ ]:
# get labels
train_data_labels = train_data.label.values
set(train_data_labels)

In [ ]:
# replace label 4 with 1
train_data_labels[train_data_labels == 4] = 1
set(train_data_labels)

### tokenization

In [35]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    train_data_clean, target_vocab_size=2**16
)

data_inputs = [tokenizer.encode(sentence) for sentence in train_data_clean]

KeyboardInterrupt: ignored

### padding

In [ ]:
MAX_LEN = max(len(sentence [for sentence in data_inputs]))
data_inputs = tf.keras.preprocessing.sequence.pad_sequences(data_inputs,
                                                            value=0,
                                                            padding="post",
                                                            maxlen=MAX_LEN)

### splitting data into train and test data

In [29]:
test_idx = np.random.randint(0, 800000, 8000)
test_idx = np.concatenate((test_idx, test_idx+800000))

In [ ]:
test_inputs = data_inputs[test_idx]
test_labels = train_data_labels[test_idx]
train_inputs = np.delete(data_inputs, test_idx, axis=0)
train_labels = np.delete(data_labels, test_idx)